## 14 같은 변수 반복 사용

### colab 접속

In [1]:
if __name__ == '__main__':
  !pip install import_ipynb
  from google.colab import drive
  drive.mount('/content/drive')
  %cd /content/drive/MyDrive/밑바닥부터 시작하는 딥러닝3

  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-cp37-none-any.whl size=2976 sha256=09f4f2bc4a6ca0c8109bfa95fa7b7fff4ccf9ce9f1e474343ee71ac5d744e59f
  Stored in directory: /root/.cache/pip/wheels/b4/7b/e9/a3a6e496115dffdb4e3085d0ae39ffe8a814eacc44bbf494b5
Successfully built import-ipynb
Mounted at /content/drive
/content/drive/MyDrive/밑바닥부터 시작하는 딥러닝3


In [2]:
import numpy as np
import import_ipynb
import numpy as np
from Deep09 import as_array


importing Jupyter notebook from Deep09.ipynb
importing Jupyter notebook from Deep08.ipynb


In [4]:
from Deep13 import Variable, add

importing Jupyter notebook from Deep13.ipynb


In [5]:
if __name__ == '__main__':
  x = Variable(np.array(3.0))
  y = add(x, x)
  print('y', y.data)

  y.backward()
  print('x.grad',x.grad)

y 6.0
x.grad 1.0


y = 2x dy/dx = 2 1로 오류?? 

### 14.1 문제의 원인

In [ ]:
class Variable:
  def __init__(self, data) :
    if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))
    self.data = data
    self.grad = None
    self.creator = None

  def set_creator(self, func):
    self.creator = func
  
  def backward(self):
    if self.grad is None:
      self.grad = np.ones_like(self.data)
    funcs = [self.creator]
    while funcs:
      f = funcs.pop()
      # output이 많으니까 차례차례 진행
      gys = [output.grad for output in f.outputs]
      gxs = f.backward(*gys)
      if not isinstance(gxs, tuple) :
        # 튜플 변환
        gxs = (gxs,)
      for x, gx in zip(f.inputs, gxs):
        ### 이부분이 원인 같은 변수를 반복사용 시 전파되는 미분 값이 덮어 써진다.
        x.grad = gx
        if x.creator is not None:
          funcs.append(x.creator)

      

### 14.2 해결책


In [8]:
class Variable:
  def __init__(self, data) :
    if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))
    self.data = data
    self.grad = None
    self.creator = None

  def set_creator(self, func):
    self.creator = func
  
  def backward(self):
    if self.grad is None:
      self.grad = np.ones_like(self.data)
    funcs = [self.creator]
    while funcs:
      f = funcs.pop()
      # output이 많으니까 차례차례 진행
      gys = [output.grad for output in f.outputs]
      gxs = f.backward(*gys)
      if not isinstance(gxs, tuple) :
        # 튜플 변환
        gxs = (gxs,)
      for x, gx in zip(f.inputs, gxs):
        ### 이부분이 원인 같은 변수를 반복사용 시 전파되는 미분 값이 덮어 써진다.
        if x.grad is None:
          x.grad = gx
        else:
          x.grad = x.grad + gx
        if x.creator is not None:
          funcs.append(x.creator)

      

In [9]:
class Function : 
  def __call__(self, *inputs):
    xs = [x.data for x in inputs]
    # 별표를 붙여 언팩
    ys = self.forward(*xs)
    # 튜플이 아닌 경우 추가 지원
    if not isinstance(ys, tuple):
      ys = (ys,)
    outputs = [Variable(as_array(y)) for y in ys ]
    for output in outputs :
      output.set_creator(self)
    self.inputs = inputs
    self.outputs = outputs
    return outputs if len(outputs) > 1 else outputs[0]
  
  def forward(self,xs):
    raise NotImplementedError()

  def backward(self,gys):
    raise NotImplementedError()

In [10]:
class Add(Function):
  def forward(self,x0,x1):
    y = x0 + x1 
    return y
  def backward(self, gy):
    return gy,gy

In [13]:
def add(x0, x1):
  return Add()(x0,x1)

In [14]:
class Square(Function):
  def forward(self,x):
    y = x**2
    return y
    
  def backward(self, gy):
    x = self.inputs[0].data
    gx = 2*x*gy
    return gx

In [15]:
def square(x):
  return Square()(x)

In [16]:
if __name__ == '__main__':
  x = Variable(np.array(3.0))
  y = add(add(x,x),x)
  y.backward()
  print(x.grad)

3.0


### 14.3 미분값 재설정

In [17]:
if __name__ == '__main__':
  x = Variable(np.array(3.0))
  y = add(x,x)
  y.backward()
  print(x.grad)

  y = add(add(x,x),x)
  y.backward()
  print(x.grad)

2.0
5.0


 + +로 해놔서 계속 중첩됨. 따라서 grad 초기화 필요. cleargrad이용

In [26]:
class Variable:
  def __init__(self, data) :
    if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))
    self.data = data
    self.grad = None
    self.creator = None

  def set_creator(self, func):
    self.creator = func
  
  def backward(self):
    if self.grad is None:
      self.grad = np.ones_like(self.data)
    funcs = [self.creator]
    while funcs:
      f = funcs.pop()
      # output이 많으니까 차례차례 진행
      gys = [output.grad for output in f.outputs]
      gxs = f.backward(*gys)
      if not isinstance(gxs, tuple) :
        # 튜플 변환
        gxs = (gxs,)
      for x, gx in zip(f.inputs, gxs):
        ### 이부분이 원인 같은 변수를 반복사용 시 전파되는 미분 값이 덮어 써진다.
        if x.grad is None:
          x.grad = gx
        else:
          x.grad = x.grad + gx
        if x.creator is not None:
          funcs.append(x.creator)

  def cleargrad(self):
    self.grad = None

In [27]:
class Function : 
  def __call__(self, *inputs):
    xs = [x.data for x in inputs]
    # 별표를 붙여 언팩
    ys = self.forward(*xs)
    # 튜플이 아닌 경우 추가 지원
    if not isinstance(ys, tuple):
      ys = (ys,)
    outputs = [Variable(as_array(y)) for y in ys ]
    for output in outputs :
      output.set_creator(self)
    self.inputs = inputs
    self.outputs = outputs
    return outputs if len(outputs) > 1 else outputs[0]
  
  def forward(self,xs):
    raise NotImplementedError()

  def backward(self,gys):
    raise NotImplementedError()

In [28]:
class Add(Function):
  def forward(self,x0,x1):
    y = x0 + x1 
    return y
  def backward(self, gy):
    return gy,gy

In [29]:
def add(x0, x1):
  return Add()(x0,x1)

In [30]:
class Square(Function):
  def forward(self,x):
    y = x**2
    return y
    
  def backward(self, gy):
    x = self.inputs[0].data
    gx = 2*x*gy
    return gx

In [31]:
def square(x):
  return Square()(x)

In [32]:
if __name__ == '__main__':
  x = Variable(np.array(3.0))
  y = add(add(x,x),x)
  y.backward()
  print(x.grad)

3.0


In [33]:
if __name__ == '__main__':
  x = Variable(np.array(3.0))
  y = add(x,x)
  y.backward()
  print(x.grad)
  x.cleargrad()
  y = add(add(x,x),x)
  y.backward()
  print(x.grad)

2.0
3.0
